In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np

In [4]:
import shutil

In [43]:
from copy import deepcopy

In [5]:
from ShallowMindBot import *
from TowerDefense import *
from TowerDefenseApi import *

In [6]:
towerDefense = TowerDefense('state.json')

In [7]:
api = TowerDefenseApi(towerDefense.getGameState())

In [8]:
bot = ShallowMindBot()

In [9]:
bot.doTurn(api)

Selected action type 4 from options [0, 1, 2, 4, 6] with probabilities [0.21018274 0.19952657 0.19744639 0.19700019 0.1958441 ]


FileNotFoundError: [Errno 2] No such file or directory: '.\\logs\\game_-1\\round_74.pkl'

In [ ]:
possible_actions = [0,1,2,6]
p, h = bot.policy_forward(bot.computeStateFeatures(api), possible_actions)

In [ ]:
p

In [ ]:
selected_action = 1

In [ ]:
h

In [ ]:
NUM_POSSIBLE_ACTIONS = 7

In [10]:
def softmax_grad(softmax):
    s = softmax.reshape(-1,1)
    return np.diagflat(s) - np.dot(s, s.T)

In [ ]:
x = np.array([float(val) for val in bot.computeStateFeatures(api).values()])

In [32]:
def gradW2(p, possible_actions, selected_action, h):
    dsoftmax = softmax_grad(p)[selected_action,:]
    dlog = dsoftmax / p[selected_action]
    # Fill in dlog with zeros at the actions that were not possible
    dlog_filled = np.zeros(NUM_ACTION_TYPES)
    dlog_filled[possible_actions] = dlog
    dW2 = dlog_filled[None,:].T.dot(h[None,:])
    return dW2, dlog_filled

In [33]:
gradW2, dlog_filled = gradW2(p, possible_actions, selected_action, h)

NameError: name 'p' is not defined

In [34]:
def gradW1(dlog_filled, W2, h, x):
    dh = W2.T.dot(dlog_filled)
    dh[h<=0] = 0
    dW1 = np.outer(dh,x)
    return dW1

In [ ]:
gradw1 = gradW1(dlog_filled, bot.model['W2'], h, x)

## Start training

In [35]:
num_batches = 1
batch_size = 100 # used to perform a RMS prop param update every batch_size steps
learning_rate = 1e-2 # learning rate used in RMS prop
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
reward_win = 10**5
reward_lose = -reward_win

In [ ]:
for batch_idx in range(num_batches):
    # Clear out the logs folder
    log_subfolders = [d for d in os.listdir(os.path.join('.','logs')) if os.path.isdir(os.path.join('.','logs',d))]
    for subfolder in log_subfolders:
        shutil.rmtree(os.path.join('.','logs',subfolder))
        
    # Run the games in the batch
    for game_idx in range(batch_size):
        ! java -jar tower-defence-runner-3.0.3.jar

In [45]:
batch_idx = 0

model_params = pickle.load(open('model_params.p','rb'))

advantages = []
win_count = 0
for game_idx in range(batch_size):
    num_rounds = max([int(f.split('.')[0].split('_')[1]) for f in os.listdir(os.path.join('.','logs','game_{}'.format(game_idx)))])
    rewards = np.zeros(num_rounds)
    for round_idx in range(num_rounds):
        with open(os.path.join('.','logs','game_{}'.format(game_idx),'round_{}.pkl'.format(round_idx)), 'rb') as f:
            state_features, possible_action_types, action_type_probabilities, hidden_layer, action_type_choice = pickle.load(f)
        rewards[round_idx] = state_features['myMinusOppScore']
        # If last round, estimate who won or lost and update rewards
        if round_idx == (num_rounds-1):
            if state_features['myMinusOppHealth'] > 0:
                rewards[round_idx] += reward_win
                win_count += 1
            else:
                rewards[round_idx] += reward_lose
    
    # Compute advantages
    advantages.append(np.cumsum(rewards[::-1])[::-1])
    
print('Won {} of {} games'.format(win_count, batch_size))
    
# Standardize advantages
advantages_all = np.hstack(advantages)
advantages_mean = np.mean(advantages_all)
advantages_std = np.std(advantages_all)

# Compute gradients
gW1 = np.zeros_like(bot.model['W1'])
gW2 = np.zeros_like(bot.model['W2'])
for game_idx in range(batch_size):
    num_rounds = max([int(f.split('.')[0].split('_')[1]) for f in os.listdir(os.path.join('.','logs','game_{}'.format(game_idx)))])
    # Increment gradients
    advantages_game = advantages[game_idx]
    for round_idx in range(num_rounds):
        with open(os.path.join('.','logs','game_{}'.format(game_idx),'round_{}.pkl'.format(round_idx)), 'rb') as f:
            state_features, possible_action_types, action_type_probabilities, hidden_layer, action_type_choice = pickle.load(f)
        action_type_choice_idx = np.where(np.array(possible_action_types)==action_type_choice)[0][0]
        gW2_round, dlog_filled = gradW2(action_type_probabilities, possible_action_types, action_type_choice_idx, hidden_layer)
        gW2 += gW2_round*((advantages_game[round_idx]-advantages_mean)/advantages_std)
        x = np.array([float(val) for val in state_features.values()])
        gW1 += gradW1(dlog_filled, model_params['W2'], hidden_layer, x)*((advantages_game[round_idx]-advantages_mean)/advantages_std)
    
# Normalize by number of games
gW1 /= batch_size
gW2 /= batch_size

# Update using gradient
new_W1 = model_params['W1'] + learning_rate*gW1
new_W2 = model_params['W2'] + learning_rate*gW2

# Save
new_model = {}
new_model['W1'] = new_W1
new_model['W2'] = new_W2
pickle.dump(new_model,open('new_model_params.p','wb'))

Won 87 of 100 games


In [49]:
new_model['W2'] - model_params['W2']

array([[ 9.41082641e-04,  6.51927495e-03,  4.11334811e-04,
         8.28630276e-03,  2.60915885e-03,  5.11966236e-03,
         5.20450526e-03,  6.56597575e-03,  5.11487042e-05,
         2.43740141e-03,  2.26086860e-04,  2.84756261e-04,
         1.33638345e-03,  1.55448610e-03,  6.25835795e-03],
       [-5.07853872e-04, -4.06885748e-03, -2.10988130e-04,
        -4.62132145e-03, -1.99593926e-03, -1.68583677e-03,
        -2.21045919e-03, -2.36700023e-03, -1.10134772e-05,
        -1.37795554e-03,  7.37375012e-05, -8.15793976e-05,
        -2.95547360e-03, -5.84644797e-04, -5.18787924e-03],
       [ 2.56561847e-03,  4.25083314e-03, -2.30839233e-04,
         1.06696587e-04,  8.84256525e-04,  7.02557352e-04,
         1.84377926e-03,  2.88809922e-04,  2.10527297e-06,
         4.30555867e-03,  2.55806576e-03,  5.92686395e-04,
         7.84332032e-03,  7.13145840e-04,  4.39236115e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000